In [ ]:
"""
Notebook: app.ipynb
Descripción general:
    Este notebook realiza la carga, preprocesamiento y análisis de datos para el modelo de predicción seleccionado para el curso Modelos y Simulación de Sistemas 1 (o MLOPS)
    Se utilizan bibliotecas como pandas y scikit-learn para transformar variables de fecha, codificar categorías
    y preparar los datos para el modelado.
    Todas las transformaciones se aplican sobre el conjunto de entrenamiento y de prueba.
Estudiantes: Davidson Arley Pérez Jiménez, Carlos Eduardo Castaño Garzón
Autor real: sanujisamarakoon
Perfil Kaggle del autor: https://www.kaggle.com/sanujisamarakoon
Competición a la que pertenece el modelo: https://www.kaggle.com/competitions/Predict-the-Customer-Satisfaction-CSE-22/overview
"""

Loading Data

In [ ]:
# Se cargan los datos de entrenamiento, prueba y el ejemplo del envío

import pandas as pd

train = pd.read_csv('train_dataset.csv')
test = pd.read_csv('test_dataset.csv')
sample_submission = pd.read_csv('sample_submission.csv')

Understanding Data

In [ ]:
# Se muestran los datos del archivo train

train

,id,user_id,age,Gender,Date_Registered,Is_current_loyalty_program_member,loyalty_points_redeemed,loyalty_tier,Received_tier_discount_percentage,Received_card_discount_percentage,...,payment_datetime,purchased_datetime,purchase_medium,final_payment,released_date,estimated_delivery_date,received_date,shipping_method,tracking_number,customer_experience
0,0,****589084,44,O,2020-01-01,NO,5,NaN,NaN,3.0,...,2020-01-05 22:27:16,2020-01-05 22:27:16,online,1293.00,2020-01-12,2020-01-17,2020-01-17,standard,***9AWDD64SYI,neutral
1,1,****494191,36,O,2020-01-04,YES,4,1.0,3.0,4.0,...,2020-01-06 00:37:51,2020-01-06 00:37:51,in-store,4522.44,2020-01-07,2020-01-12,2020-01-09,express,***3SSRORRZ0X,bad
2,2,****216469,40,F,2020-01-02,NO,3,NaN,NaN,3.0,...,2020-01-07 03:02:35,2020-01-07 03:02:35,online,5628.00,2020-01-12,2020-01-18,2020-01-18,express,***2VSB7MH7FN,good
3,3,****707170,33,M,2020-01-06,YES,1,1.0,3.0,2.0,...,2020-01-09 22:05:39,2020-01-09 22:05:39,in-store,2073.32,2020-01-13,2020-01-15,2020-01-17,express,***9XENHE2PKZ,bad
4,4,****066329,43,O,2020-01-06,YES,1,1.0,3.0,4.0,...,2020-01-11 08:32:22,2020-01-11 08:32:22,online,310.86,2020-01-15,2020-01-22,2020-01-20,express,***QTWLMEL0PE,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206964,206964,****708759,19,M,2024-12-18,NO,4,NaN,NaN,3.0,...,2024-12-28 23:51:36,2024-12-28 23:51:36,online,5533.00,2024-12-31,2025-01-02,2024-12-31,standard,***OD1GASFD9M,bad
206965,206965,****074598,54,M,2024-12-27,NO,2,NaN,NaN,1.0,...,2024-12-28 23:54:24,2024-12-28 23:54:24,online,4910.00,2024-12-30,2025-01-01,2025-01-02,standard,***TGSHGSXIL3,bad
206966,206966,****016861,25,O,2024-12-02,YES,4,4.0,10.0,4.0,...,2024-12-28 23:57:17,2024-12-28 23:57:17,in-store,4558.99,2025-01-04,2025-01-11,2025-01-10,express,***0C2DHORJKD,bad
206967,206967,****741534,54,M,2024-12-12,NO,0,NaN,NaN,NaN,...,2024-12-28 23:57:28,2024-12-28 23:57:28,online,8900.00,2024-12-30,2025-01-04,2025-01-06,standard,***AXVOZIFGOZ,good


In [ ]:
# Se muestra el tipo de dato de cada columna del train

train.dtypes

,0
id,int64
user_id,object
age,int64
Gender,object
Date_Registered,object
Is_current_loyalty_program_member,object
loyalty_points_redeemed,int64
loyalty_tier,float64
Received_tier_discount_percentage,float64
Received_card_discount_percentage,float64


In [ ]:
# Se muestra la cantidad de valores nulos de cada columna

train.isnull().sum()

,0
id,0
user_id,0
age,0
Gender,0
Date_Registered,0
Is_current_loyalty_program_member,0
loyalty_points_redeemed,0
loyalty_tier,102067
Received_tier_discount_percentage,103379
Received_card_discount_percentage,156879


In [ ]:
# Se muestra la cantidad de cada tipo de experiencia en el conjunto de datos

train['customer_experience'].value_counts()

,count
customer_experience,
good,91388
bad,68538
neutral,47043


Data Cleaning

In [ ]:
# Se convierten las columnas de fecha al formato datetime

train['Date_Registered'] = pd.to_datetime(train['Date_Registered'])
train['payment_datetime'] = pd.to_datetime(train['payment_datetime'])
train['purchased_datetime'] = pd.to_datetime(train['purchased_datetime'])
train['released_date'] = pd.to_datetime(train['released_date'])
train['estimated_delivery_date'] = pd.to_datetime(train['estimated_delivery_date'])
train['received_date'] = pd.to_datetime(train['received_date'])

In [ ]:
# Se muestra que las columnas de fechas ahora son del tipo datetime

train.dtypes

,0
id,int64
user_id,object
age,int64
Gender,object
Date_Registered,datetime64[ns]
Is_current_loyalty_program_member,object
loyalty_points_redeemed,int64
loyalty_tier,float64
Received_tier_discount_percentage,float64
Received_card_discount_percentage,float64


Adding new features

In [ ]:
# Se extraen las características temporales y se generan variables derivadas de las fechas (hora, día, mes, etc.)

train['purchase_hour'] = train['payment_datetime'].dt.hour
train['purchase_day'] = train['payment_datetime'].dt.day_name()
train['purchase_month'] = train['payment_datetime'].dt.month
train['days_since_registration'] = (train['purchased_datetime'] - train['Date_Registered']).dt.days
train['estimated_delivery_day'] = train['estimated_delivery_date'].dt.day_name()
train['received_day'] = train['received_date'].dt.day_name()

In [ ]:
# Se convierten las columnas de fecha a timestamp de Unix

date_columns = ['Date_Registered', 'payment_datetime', 'purchased_datetime',
                'released_date', 'estimated_delivery_date', 'received_date']

for col in date_columns:
    train[col] = pd.to_datetime(train[col], errors='coerce').astype(int) / 10**9

Categorical Encoding

In [ ]:
# Se convierte la variable customer_experience a valores númericos ordenados

train['customer_experience'] = pd.Categorical(train['customer_experience'], categories=['bad', 'neutral', 'good'], ordered=True)
train['customer_experience'] = train['customer_experience'].cat.codes

One Hot Encoding

In [ ]:
# Se convierten las variables categóricas a un formato númerico

from sklearn.preprocessing import LabelEncoder

categorical_columns = [ 'Gender', 'Is_current_loyalty_program_member',
                       'product_category',
                       'payment_method', 'purchase_medium', 'shipping_method', 'purchase_day','estimated_delivery_day','received_day']

label_encoder = LabelEncoder()

for col in categorical_columns:
    train[col] = label_encoder.fit_transform(train[col].astype(str))

Adding more new features

In [ ]:
# Se crean nuevas variables derivadas de las fechas y variables económicas

train['Delivery_time'] = train['received_date'] - train['released_date']
train['Delivery_delay'] = train['received_date'] - train['estimated_delivery_date']
train['Waiting_time'] = train['received_date'] - train['payment_datetime']
train['Additional_charge'] = train['final_payment'] - train['Product_value']
train['Waiting_percentage'] = (train['received_date'] - train['estimated_delivery_date'])/(train['received_date'] - train['payment_datetime'])
train['Processing_time'] = train['released_date'] - train['payment_datetime']
train['Loyalty_engagement'] = train['loyalty_points_redeemed'] / train['Product_value']

In [ ]:
# Convierte los valores no numéricos a valores nulos

import numpy as np

train.replace(r'[^0-9]+', np.nan, regex=True, inplace=True)

/tmp/ipython-input-2202553812.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train.replace(r'[^0-9]+', np.nan, regex=True, inplace=True)


In [ ]:
# Cambia los valores faltantes por el número 0 y se convierte todo el Data Frame a tipo numérico

train.fillna(0, inplace=True)
train = train.apply(pd.to_numeric)

In [ ]:
# Se separan las variables predictoras y la variable objetivo
# X: variables independientes (todas las columnas excepto 'customer_experience')
# y: variable dependiente u objetivo ('customer_experience'), es la que se va a predecir

X = train.drop('customer_experience', axis=1)
y = train['customer_experience']

In [ ]:
# Se hace un análisis de relevancia de las variable predictoras
# Se utiliza la métrica de Información Mutua (Mutual Information) para evaluar la dependencia entre cada variable predictora y la variable objetivo
# Cuanto mayor sea el MI Score, más información aporta esa variable al modelo.

from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(X, y,random_state=42)
mi_scores_df = pd.DataFrame({'Feature': X.columns, 'MI Score': mi_scores})

print(mi_scores_df.sort_values(by='MI Score', ascending=False))

                                Feature  MI Score
30                         received_day  0.063168
34                    Additional_charge  0.062853
22                        received_date  0.062619
33                         Waiting_time  0.057707
28              days_since_registration  0.037840
35                   Waiting_percentage  0.035122
31                        Delivery_time  0.027084
36                      Processing_time  0.013077
4                       Date_Registered  0.012623
32                       Delivery_delay  0.012315
23                      shipping_method  0.012256
5     Is_current_loyalty_program_member  0.012225
18                      purchase_medium  0.010702
9     Received_card_discount_percentage  0.008956
10  Received_coupon_discount_percentage  0.007978
3                                Gender  0.007529
2                                   age  0.007195
7                          loyalty_tier  0.006055
6               loyalty_points_redeemed  0.004186


Dropping least important columns

In [ ]:
# Se eliminan las variables que no son relevantes para la predicción (variables con MI Score más cercano a 0)

X = train.drop(['customer_experience','tracking_number', 'user_id', 'loyalty_tier','purchase_medium' ,'shipping_method','Gender','order_id', 'Received_tier_discount_percentage','Is_current_loyalty_program_member', 'transaction_id'],axis=1)
y = train['customer_experience']

In [ ]:
# Se separa el dataset en dos subconjuntos: uno para entrenar el modelo (85% de los datos) y otro para evaluar su rendimiento (15% de los datos)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

Training the model

In [ ]:
# Se configura un Random Forest Classifier como modelo predictivo

from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier(class_weight='balanced', random_state=42)
# model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, class_weight='balanced')
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    class_weight='balanced'
)

In [ ]:
# # Define a list of hyperparameter combinations
# param_list = [
#     {'n_estimators': 100, 'max_depth': 10},
#     {'n_estimators': 200, 'max_depth': None},
#     {'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 5},
#     {'n_estimators': 100, 'max_depth': 15, 'min_samples_split': 10}
# ]

# best_model = None
# best_score = 0

# # Iterate through parameters and test each combination
# for params in param_list:
#     model = RandomForestClassifier(**params, class_weight='balanced', random_state=42)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     score = accuracy_score(y_test, y_pred)
#     print(f"Params: {params}, Accuracy: {score}")
#     if score > best_score:
#         best_model = model
#         best_score = score

# print("Best Parameters:", best_model.get_params())

In [ ]:
# Se entrena el modelo de Random Forest con los datos de entrenamiento

model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', min_samples_leaf=2,
                       min_samples_split=5, n_estimators=300, random_state=42)

In [ ]:
# Se realiza la predicción con los datos de prueba

y_pred = model.predict(X_test)

In [ ]:
# Se miden las métricas de rendimiento del modelo Random Forest sobre el conjunto de prueba

from sklearn.metrics import classification_report, accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.675030599755202
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.67      0.63     10285
           1       0.75      0.69      0.72      7130
           2       0.72      0.67      0.69     13631

    accuracy                           0.68     31046
   macro avg       0.69      0.68      0.68     31046
weighted avg       0.68      0.68      0.68     31046



In [ ]:
# Se evalúa el modelo Random Forest con la métrica planteada en la competición: Weighted F1 Score

from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred, average='weighted')
print(f'Weighted F1 Score: {f1}')

Weighted F1 Score: 0.6768735948449933


In [ ]:
# import optuna
# from lightgbm import LGBMClassifier
# from sklearn.model_selection import cross_val_score

# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#         'max_depth': trial.suggest_int('max_depth', -1, 15),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
#         'num_leaves': trial.suggest_int('num_leaves', 31, 100),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
#         'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
#         'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
#     }

#     model = LGBMClassifier(random_state=42, **params)
#     scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
#     return scores.mean()

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=20)

# print("Best Parameters:", study.best_params)
# print("Best Score:", study.best_value)

In [ ]:
# Se prueba otro modelo con el LGBMC Classifier

from lightgbm import LGBMClassifier

model = LGBMClassifier(
    n_estimators=100,
    max_depth=9,
    learning_rate=0.1,
    num_leaves=57,
    feature_fraction=0.7233,
    bagging_fraction=0.7492,
    lambda_l1=1.9796,
    lambda_l2=8.1072,
    random_state=42
)

In [ ]:
# Se entrena el modelo de LGBM con los datos de prueba

model.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.7233, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7233
[LightGBM] [Warning] lambda_l2 is set=8.1072, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.1072
[LightGBM] [Warning] lambda_l1 is set=1.9796, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.9796
[LightGBM] [Warning] bagging_fraction is set=0.7492, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7492
[LightGBM] [Warning] feature_fraction is set=0.7233, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7233
[LightGBM] [Warning] lambda_l2 is set=8.1072, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.1072
[LightGBM] [Warning] lambda_l1 is set=1.9796, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.9796
[LightGBM] [Warning] bagging_fraction is set=0.7492, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7492
[LightGBM] [Info] Auto-choosing col-wise multi-threading

LGBMClassifier(bagging_fraction=0.7492, feature_fraction=0.7233,
               lambda_l1=1.9796, lambda_l2=8.1072, max_depth=9, num_leaves=57,
               random_state=42)

In [ ]:
# Se realiza la predicción con los datos de prueba

y_pred = model.predict(X_test)

[LightGBM] [Warning] feature_fraction is set=0.7233, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7233
[LightGBM] [Warning] lambda_l2 is set=8.1072, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.1072
[LightGBM] [Warning] lambda_l1 is set=1.9796, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.9796
[LightGBM] [Warning] bagging_fraction is set=0.7492, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7492


In [ ]:
# Se miden las métricas de rendimiento del modelo LGBM sobre el conjunto de prueba

from sklearn.metrics import classification_report, accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6866585067319462
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.71      0.64     10285
           1       0.82      0.68      0.74      7130
           2       0.72      0.67      0.70     13631

    accuracy                           0.69     31046
   macro avg       0.71      0.69      0.69     31046
weighted avg       0.70      0.69      0.69     31046



In [ ]:
# Se evalúa el modelo LGMB con la métrica planteada en la competición: Weighted F1 Score
# Se obtiene un mayor F1 Score, por lo que se escogerá este modelo para el conjunto de datos de test

from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred, average='weighted')
print(f'Weighted F1 Score: {f1}')

Weighted F1 Score: 0.6894167185240283


In [ ]:
# Se muestran los datos del archivo test

test

,id,user_id,age,Gender,Date_Registered,Is_current_loyalty_program_member,loyalty_points_redeemed,loyalty_tier,Received_tier_discount_percentage,Received_card_discount_percentage,...,payment_method,payment_datetime,purchased_datetime,purchase_medium,final_payment,released_date,estimated_delivery_date,received_date,shipping_method,tracking_number
0,0,****897735,30,F,2020-01-05,YES,4,4.0,10.0,NaN,...,mastercard_d,2020-01-06 07:17:59,2020-01-06 07:17:59,online,8192.0,2020-01-13,2020-01-15,2020-01-17,standard,***AKDCDBBOVK
1,1,****386832,57,O,2020-01-06,NO,4,NaN,NaN,NaN,...,cash,2020-01-08 10:17:58,2020-01-08 10:17:58,online,8035.0,2020-01-15,2020-01-20,2020-01-17,standard,***ZNIUIC3ZXT
2,2,****320205,35,O,2020-01-05,NO,0,NaN,NaN,NaN,...,mastercard_d,2020-01-10 03:44:20,2020-01-10 03:44:20,in-store,9896.0,2020-01-12,2020-01-18,2020-01-18,standard,***1G0TUWLUQO
3,3,****423059,36,M,2020-01-01,YES,4,4.0,10.0,NaN,...,grabpay,2020-01-12 23:37:09,2020-01-12 23:37:09,online,3419.0,2020-01-15,2020-01-22,2020-01-24,express,***HFH0DHDMVR
4,4,****453616,21,O,2020-01-06,YES,3,2.0,5.0,NaN,...,cash,2020-01-13 00:23:02,2020-01-13 00:23:02,in-store,8081.0,2020-01-15,2020-01-21,2020-01-23,express,***EM0MLB4NAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137966,137966,****565215,57,F,2024-12-24,YES,3,1.0,3.0,NaN,...,otc,2024-12-28 23:50:44,2024-12-28 23:50:44,online,2805.0,2024-12-29,2024-12-30,2025-01-01,standard,***YNJDKIJLK9
137967,137967,****023402,34,O,2024-12-20,YES,1,4.0,10.0,NaN,...,otc,2024-12-28 23:55:19,2024-12-28 23:55:19,in-store,224.0,2024-12-29,2025-01-02,2025-01-02,standard,***L3Y7UFTDQX
137968,137968,****101693,42,O,2023-07-15,YES,0,1.0,3.0,NaN,...,coinsph,2024-12-28 23:55:21,2024-12-28 23:55:21,in-store,2392.0,2025-01-03,2025-01-06,2025-01-08,express,***OLT1FFKXSG
137969,137969,****117354,51,O,2024-12-12,NO,2,NaN,NaN,5.0,...,amex,2024-12-28 23:58:40,2024-12-28 23:58:40,in-store,2345.0,2024-12-30,2025-01-02,2025-01-03,standard,***HDJFMBF0LX


In [ ]:
# Se convierten las columnas de fecha al formato datetime

test['Date_Registered'] = pd.to_datetime(test['Date_Registered'])
test['payment_datetime'] = pd.to_datetime(test['payment_datetime'])
test['purchased_datetime'] = pd.to_datetime(test['purchased_datetime'])
test['released_date'] = pd.to_datetime(test['released_date'])
test['estimated_delivery_date'] = pd.to_datetime(test['estimated_delivery_date'])
test['received_date'] = pd.to_datetime(test['received_date'])

In [ ]:
# Se extraen las características temporales y se generan variables derivadas de las fechas (hora, día, mes, etc.)

test['purchase_hour'] = test['payment_datetime'].dt.hour
test['purchase_day'] = test['payment_datetime'].dt.day_name()
test['purchase_month'] = test['payment_datetime'].dt.month
test['days_since_registration'] = (test['purchased_datetime'] - test['Date_Registered']).dt.days
test['estimated_delivery_day'] = test['estimated_delivery_date'].dt.day_name()
test['received_day'] = test['received_date'].dt.day_name()

In [ ]:
# Se convierten las columnas de fecha a timestamp de Unix

date_columns = ['Date_Registered', 'payment_datetime', 'purchased_datetime',
                'released_date', 'estimated_delivery_date', 'received_date']

for col in date_columns:
    test[col] = pd.to_datetime(test[col], errors='coerce').astype(int) / 10**9

In [ ]:
# Se convierten las variables categóricas a un formato númerico

from sklearn.preprocessing import LabelEncoder

categorical_columns = [ 'Gender', 'Is_current_loyalty_program_member',
                       'product_category',
                       'payment_method', 'purchase_medium', 'shipping_method', 'purchase_day','estimated_delivery_day','received_day']

label_encoder = LabelEncoder()

for col in categorical_columns:
    test[col] = label_encoder.fit_transform(test[col].astype(str))

In [ ]:
# Se crean nuevas variables derivadas de las fechas y variables económicas

test['Delivery_time'] = test['received_date'] - test['released_date']
test['Delivery_delay'] = test['received_date'] - test['estimated_delivery_date']
test['Waiting_time'] = test['received_date'] - test['payment_datetime']
test['Additional_charge'] = test['final_payment'] - test['Product_value']
test['Waiting_percentage'] = (test['received_date'] - test['estimated_delivery_date'])/(test['received_date'] - test['payment_datetime'])
test['Processing_time'] = test['released_date'] - test['payment_datetime']
test['Loyalty_engagement'] = test['loyalty_points_redeemed'] / test['Product_value']

In [ ]:
# Convierte los valores no numéricos a valores nulos

test.replace(r'[^0-9]+', np.nan, regex=True, inplace=True)

/tmp/ipython-input-2946675442.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test.replace(r'[^0-9]+', np.nan, regex=True, inplace=True)


In [ ]:
# Cambia los valores faltantes por el número 0

test.fillna(0,inplace=True)

In [ ]:
# Se convierte todo el Data Frame a tipo numérico

test = test.apply(pd.to_numeric)

In [ ]:
# Ya sabiendo cuales fueron las variables con el MI Score más bajo, se eliminan estas del conjunto de datos de test

new_test=test.drop(['tracking_number', 'user_id', 'loyalty_tier','purchase_medium' ,'shipping_method','Gender','order_id', 'Received_tier_discount_percentage', 'Is_current_loyalty_program_member', 'transaction_id'], axis=1)

In [ ]:
# Se prueba finalmente el modelo con el LGBMC Classifier

final_model = LGBMClassifier(
    n_estimators=100,
    max_depth=9,
    learning_rate=0.1,
    num_leaves=57,
    feature_fraction=0.7233,
    bagging_fraction=0.7492,
    lambda_l1=1.9796,
    lambda_l2=8.1072,
    random_state=42
)

In [ ]:
# Se entrena el modelo de LGBM

final_model.fit(X, y)

[LightGBM] [Warning] feature_fraction is set=0.7233, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7233
[LightGBM] [Warning] lambda_l2 is set=8.1072, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.1072
[LightGBM] [Warning] lambda_l1 is set=1.9796, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.9796
[LightGBM] [Warning] bagging_fraction is set=0.7492, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7492
[LightGBM] [Warning] feature_fraction is set=0.7233, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7233
[LightGBM] [Warning] lambda_l2 is set=8.1072, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.1072
[LightGBM] [Warning] lambda_l1 is set=1.9796, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.9796
[LightGBM] [Warning] bagging_fraction is set=0.7492, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7492
[LightGBM] [Info] Auto-choosing row-wise multi-threading

LGBMClassifier(bagging_fraction=0.7492, feature_fraction=0.7233,
               lambda_l1=1.9796, lambda_l2=8.1072, max_depth=9, num_leaves=57,
               random_state=42)

In [ ]:
# Se realiza la predicción con el modelo de LGBM Classifier

y_pred = final_model.predict(new_test)

[LightGBM] [Warning] feature_fraction is set=0.7233, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7233
[LightGBM] [Warning] lambda_l2 is set=8.1072, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.1072
[LightGBM] [Warning] lambda_l1 is set=1.9796, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.9796
[LightGBM] [Warning] bagging_fraction is set=0.7492, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7492


In [ ]:
# Se genera el archivo de predicciones finales en el formato definido para la competición
# Se convierten las predicciones numéricas del modelo en etiquetas textuales interpretables ("bad", "neutral", "good") y exporta los resltados en el archivo submission.csv

label_mapping = {0: "bad", 1: "neutral", 2: "good"}
y_pred_labels = [label_mapping[label] for label in y_pred]

submission = pd.DataFrame({"id": new_test["id"], "customer_experience": y_pred_labels})
submission.to_csv("submission.csv", index=False)

In [ ]:
# Se lee el archivo final exportado

submission = pd.read_csv("submission.csv")

In [ ]:
# Se muestra el archivo en el formato predefinido para la competición

submission

,id,customer_experience
0,0,neutral
1,1,neutral
2,2,good
3,3,bad
4,4,bad
...,...,...
137966,137966,bad
137967,137967,bad
137968,137968,neutral
137969,137969,bad
